In [31]:
import os

import numpy as np
import pandas as pd

from utils import data_dir

dataset = pd.read_csv(os.path.join(data_dir, 'VDem_13_urbpop.csv'))
stata_data = pd.read_csv(os.path.join(data_dir, 'Stata-postprocess-2.csv'))

C:\Users\Etienne\AppData\Local\Temp\ipykernel_8988\1825453167.py:8: DtypeWarning: Columns (395,832,833,931,1236,3283,3284,3352,3354,3358,3359,3361,3362,3363) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(os.path.join(data_dir, 'VDem_13_urbpop.csv'))
C:\Users\Etienne\AppData\Local\Temp\ipykernel_8988\1825453167.py:9: DtypeWarning: Columns (395,832,833,931,1236,3283,3284,3352,3354,3358,3359,3361,3362,3363) have mixed types. Specify dtype option on import or set low_memory=False.
  stata_data = pd.read_csv(os.path.join(data_dir, 'Stata-postprocess-2.csv'))


In [32]:
def isin(c):
    print(c, c in stata_data.columns)

isin('p_GoodsRatio_WmP')
isin('GoodsRatio')
isin('ratiopublicprivate')

p_GoodsRatio_WmP True
GoodsRatio True
ratiopublicprivate False


In [33]:
import numpy as np

def linear_log_likelihood(y_obs, y_pred):
    """  """
    residuals = y_obs - y_pred
    sigma_squared = np.var(residuals)
    n = len(y_obs)

    log_likelihood = (
        - 0.5 * n * np.log(2 * np.pi) 
        - 0.5 * n * np.log(sigma_squared) 
        - 0.5 * np.sum(residuals ** 2) / sigma_squared
    )
    return log_likelihood

models = ['WmP', 'WmE', 'WmU', 'W4']
controls = ['e_migdppcln', 'logpop']

def restrict_data(*outcomes: str):
    filtered = stata_data
    for col in (*outcomes, *models, *controls,):
        filtered = filtered[~filtered[col].isna()]
    return filtered

f = restrict_data('CivilLiberties')

ll_m = linear_log_likelihood(f['CivilLiberties'], f['p_CivilLiberties_WmU'])
ll_n = linear_log_likelihood(f['CivilLiberties'], f['CivilLiberties'].mean())

print(ll_m)
print(ll_n)

2743.7634689975966
-2323.6933119329433


In [34]:
len(f)

7399

In [37]:
print(f['CivilLiberties'].max(), f['CivilLiberties'].min())
r = f['CivilLiberties']-f['p_CivilLiberties_WmU']
print(r)
print(np.var(r))
print(np.std(r))
print(np.mean(r))
print(f[['CivilLiberties', 'p_CivilLiberties_WmU']])

0.986 0.0089999996
0       -0.058953
1       -0.053342
2       -0.056637
3       -0.059332
4       -0.065929
           ...   
10133   -0.087122
10134   -0.087733
10135   -0.089194
10136   -0.088220
10137   -0.097797
Length: 7399, dtype: float64
0.02788866389369014
0.16699899369065113
1.0325761840078486e-11
       CivilLiberties  p_CivilLiberties_WmU
0               0.401              0.459953
1               0.410              0.463342
2               0.410              0.466637
3               0.410              0.469332
4               0.405              0.470929
...               ...                   ...
10133           0.799              0.886122
10134           0.799              0.886733
10135           0.723              0.812194
10136           0.668              0.756220
10137           0.656              0.753797

[7399 rows x 2 columns]


In [40]:
from scipy.stats import norm

density = norm.pdf(r, loc=np.mean(r), scale=np.std(r))

In [45]:
np.sum(np.log(density))

2743.7634689975976